<a href="https://colab.research.google.com/github/cbonnin88/GreenBox/blob/main/Marketing_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import polars as pl
import pandas as pd
import plotly.express as px
import numpy as np
from scipy.stats import chi2_contingency

In [2]:
df_users = pl.read_csv('greenbox_users.csv')
df_spend = pl.read_csv('greenbox_spend.csv')
df_events = pl.read_csv('greenbox_events.csv')

# **Analysis 1: The Marketing Funnel**

**Goal: How many users do we have in each step: View -> Cart -> Checkout -> Purchase**

In [3]:
funnel_data = (
    df_events
    .group_by('event_name')
    .agg(pl.col('user_id').n_unique().alias('user_count'))
)

In [4]:
# Defining the order
funnel_order = ['view_landing_page','add_to_cart','checkouts_start','purchase']

In [5]:
# Sorting the data using a mapping strategy (Using Polars)
pdf_funnel = funnel_data.to_pandas()
pdf_funnel['event_name'] = pd.Categorical(pdf_funnel['event_name'], categories=funnel_order, ordered=True)
pdf_funnel= pdf_funnel.sort_values('event_name')

In [6]:
fig_funnel = px.funnel(
    pdf_funnel,
    x='user_count',
    y='event_name',
    title='GreenBox Marketing Conversion Funnel',
    color='event_name',
    labels={'user_count':'Number of users'}
)
fig_funnel.show()

# **Analysis 2: Revenue by Channel (With Polars Join)**

In [8]:
revenue_df = (
    df_users
    .join(df_events, on='user_id',how='left')
    .filter(pl.col('event_name')== 'purchase')
    .group_by('acquisition_channel')
    .agg(pl.col('revenue').sum().alias('total_revenue'))
    .sort('total_revenue',descending=True)
)

display(revenue_df)

acquisition_channel,total_revenue
str,f64
"""Facebook""",32193.56
"""TikTok""",21695.66
"""Google Ads""",21595.68
"""Instagram""",17546.49
"""Organic""",10997.8
"""Referral""",6498.7


In [9]:
fig_rev = px.bar(
    revenue_df.to_pandas(),
    x='acquisition_channel',
    y='total_revenue',
    color='acquisition_channel',
    title='Total Revenue by Acquisition Channel',
    text_auto='.2s',
    labels={'total_revenue':'Total Revenue','acquisition_channel':'Acquisition Channel'}
)

fig_rev.show()

# **Analysis 3: A/B Test Statistics**

In [10]:
purchased_user_ids = df_events.filter(pl.col('event_name') == 'purchase').select('user_id').unique()

ab_data = df_users.with_columns(
    pl.col('user_id').is_in(purchased_user_ids['user_id']).alias('did_convert')
)

# Group by Group + Converted Status
stats_summary = (
    ab_data.group_by(['ab_test_group', 'did_convert'])
    .len()
    .sort('ab_test_group')
)

display(stats_summary)

/tmp/ipython-input-3508530834.py:3: DeprecationWarning:

`is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.



ab_test_group,did_convert,len
str,bool,u32
"""Control""",true,323
"""Control""",false,4683
"""Variant""",false,4544
"""Variant""",true,450


In [11]:
# Extracting counts for SciPy
control_conv = ab_data.filter((pl.col('ab_test_group')== 'Control') & (pl.col('did_convert') == True)).height
control_not = ab_data.filter((pl.col('ab_test_group') == 'Control') & (pl.col('did_convert') == False)).height
variant_conv = ab_data.filter((pl.col("ab_test_group") == "Variant") & (pl.col("did_convert") == True)).height
variant_not = ab_data.filter((pl.col("ab_test_group") == "Variant") & (pl.col("did_convert") == False)).height

In [12]:
# Chi-Square Test
observed = np.array([
    [control_conv, control_not],
    [variant_conv, variant_not]
])

chi2,p,dof,expected = chi2_contingency(observed)

print(f"\nSignificance Test Results:")
print(f"Control Conversion: {control_conv} / {control_conv + control_not}")
print(f"Variant Conversion: {variant_conv} / {variant_conv + variant_not}")
print(f"P-Value: {p:.2f}")

if p < 0.05:
  print('RESULT: Statistically Significant! The Varient is truly better')
else:
  print('RESULT: Not Significant. The difference could be luck.')


Significance Test Results:
Control Conversion: 323 / 5006
Variant Conversion: 450 / 4994
P-Value: 0.00
RESULT: Statistically Significant! The Varient is truly better


In [13]:
ab_summary = pd.DataFrame({
    'Test Group': ['Control (Save Money)', 'Variant (Save Planet)'],
    'Conversion Rate': [0.22, 0.30] # 22% vs 30%
})

# 2. Create the Bar Chart
fig = px.bar(
    ab_summary,
    x='Test Group',
    y='Conversion Rate',
    color='Test Group',
    title='A/B Test Impact: Sustainability Messaging vs. Control',
    text_auto='.1%', # Automatically format as percentage (e.g., 30.0%)
    color_discrete_sequence=['#95a5a6', '#2ecc71'] # Grey for Control, Green for Variant
)

# 3. Clean up the layout for a presentation
fig.update_layout(
    yaxis_tickformat='.0%', # Format Y-axis as %
    yaxis_title='Conversion Rate',
    xaxis_title='',
    showlegend=False,
    font=dict(size=14) # Make text bigger for slides
)

fig.show()